In [13]:
%matplotlib inline
%precision 16
import numpy
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
import matplotlib.pyplot as plt

# The Alternative Directiron Implicit(ADI) Scheme

In previous implementations, large number of iterations are needed in each time steps, thus decrease the efficiency of computation. One alternative is to use a partially implicit method which preserves the tridiagonal structure of the matrix for the implicit terms. This construction is called the alternative direction implicit (ADI) scheme.

Similarly, the fully implicit scheme is based on the finite difference methods of the Heston PDE:

$$\begin{aligned}
\frac{U^{n+1}_{i,j} - U^n_{i,j}}{\Delta t} =& \left[ S_i^2v_j \frac{U^n_{i+1,j} - 2U^n_{i,j} + U_{i-1,j}}{2\Delta S^2} + \rho\sigma S_iv_j \frac{U^n_{i+1,j+1} + U^n_{i-1,j-1} - U^n_{i-1,j+1} - U_{i+1,j-1}}{4\Delta S\Delta v}\right. \\
&\left. + \sigma^2v_j\frac{U^n_{i,j+1} - 2U_{i,j} + U_{i,j-1}}{2\Delta v^2} + rS\frac{U^n_{i+1,j} - U^n_{i-1,j}}{2\Delta S} + \kappa(\theta - v_j)\frac{U^n{i,j+1} - U^n_{i,j-1}}{2\Delta v}-rU^n_{i,j}\right]
\end{aligned}$$

which can be re-written as:
$$
a_{i,j}U^{n+1}_{i,j} + c_{i,j}U^{n+1}_{i-1,j} + d_{i,j}U^{n+1}_{i+1,j} + e_{i,j}U^{n+1}_{i,j-1} + f_{i,j}U^{n+1}_{i,j+1} + b_{i,j}(U^{n+1}_{i-1,j-1} - U^{n+1}_{i-1,j+1} - U^{n+1}_{i+1,j-1} + U^{n+1}_{i+1,j+1}) = U^{n}_{i,j}
$$

In [14]:
K = 100.0
r = 0.03
kappa = 2.0
theta = 0.20
sigma = 0.3
rho = -0.5
_lambda = 0.0

S_min = 0
S_max = 200

V_min = 0.0
V_max = 1.0

t_0 = 0.0
t_final = 1.0

In [15]:
M = 100
N = 100
L = 100

s  = numpy.linspace(S_min, S_max, N + 1) 
delta_s = s[1] - s[0]
v = numpy.linspace(V_min, V_max, M + 1)
delta_v = v[1] - v[0]
t = numpy.linspace(t_0, t_final, L + 1)
delta_t = t[1] - t[0]
print delta_s
print delta_v
print delta_t

2.0
0.01
0.01


In [16]:
# construct Matrix A_0, A_1 and A_2
from scipy.sparse import spdiags
R_s = delta_t / delta_s
R_v = delta_t / delta_v
R_ss = delta_t / delta_s**2
R_vv = delta_t / delta_v**2
R_sv = delta_t / (delta_s * delta_v)

# A1
A_1 = numpy.zeros((N+1, N+1))
A_11 = numpy.zeros((N+1, N+1))
A_12 = numpy.zeros((N+1, N+1))
# main diagonal entries
diag1 = numpy.zeros(N+1)
for i in xrange(N+1):
    diag1[i] = - (s[0] + i * delta_s)**2 / delta_s**2
# sub-diagonal entries
sub1 = numpy.zeros(N+1)
for i in xrange(N+1):
    sub1[i] = (s[1] + i * delta_s)**2 / (2.0 * delta_s**2)
sub2 = numpy.zeros(N+1)
for i in xrange(2, N+1):
    sub2[i] = (s[0] + (i-1) * delta_s)**2 / (2.0 * delta_s**2)
    
A_11 = sparse.spdiags([sub1, diag1, sub2], [-1, 0, 1], N + 1, N + 1)

# main diagonal entries
diag2 = - 0.5 * r * numpy.ones(N+1)
# sub-diagonal entries
_sub1 = numpy.zeros(N+1)
for i in xrange(N+1):
    _sub1[i] = - r * (s[1] + i * delta_s) / (2.0 * delta_s)
_sub2 = numpy.zeros(N+1)
for i in xrange(2,N+1):
    _sub2[i] = r * (s[0] + (i-1) * delta_s) / (2.0 * delta_s)

A_12 = sparse.spdiags([_sub1, diag2, _sub2], [-1, 0, 1], N + 1, N + 1)

#A2
A_2 = numpy.zeros((M+1, M+1))
# main diagonal entries
e = numpy.zeros(M+1)
for i in xrange(1,M+1):
    e[i] = - _lambda**2 * (v[0] + i * delta_v) / delta_v**2 - 0.5 * r
d1 = numpy.zeros(M+1)
for i in xrange(1,M+1):
    d1[i] = - _lambda**2 * (v[1] + i * delta_v) / (2 * delta_v**2) - kappa * (theta - (v[1] + i * delta_v)) / (2.0 * delta_v)
d2 = numpy.zeros(M+1)
for i in xrange(2,M+1):
    d2[i] = - _lambda**2 * (v[0] + (i-1) * delta_v) / (2 * delta_v**2) + kappa * (theta - (v[1] + i * delta_v)) / (2.0 * delta_v)
    
A_2 = sparse.spdiags([d1, e, d2], [-1, 0, 1], M + 1, M + 1)